<a href="https://colab.research.google.com/github/SanjidaJahanTanha/Annual-SaleReport/blob/main/direct_access_medical_imaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install Kaggle API and Setup Kaggle API Key
from google.colab import files
files.upload()  # This will prompt you to upload the kaggle.json file

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# Download the Dataset Directly from Kaggle
!kaggle competitions download -c rsna-pneumonia-detection-challenge -p /content
!unzip -q /content/rsna-pneumonia-detection-challenge.zip -d /content/rsna-pneumonia-detection-challenge

rsna-pneumonia-detection-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)
replace /content/rsna-pneumonia-detection-challenge/GCP Credits Request Link - RSNA.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# verify if the dataset was extracted correctly
import os

# Expected file path
img_path = '/content/rsna-pneumonia-detection-challenge/stage_2_train_images/0004cfab-14fd-4e49-80ba-63afce7341b9.dcm'

# Check if the file exists
if os.path.exists(img_path):
    print(f"File found at: {img_path}")
else:
    print(f"File NOT found at: {img_path}")
    print("Checking if the directory exists...")

    # Check if the directory exists
    dir_path = '/content/rsna-pneumonia-detection-challenge/stage_2_train_images/'
    if os.path.exists(dir_path):
        print(f"Directory found: {dir_path}")
        print("List of files in the directory:")
        !ls {dir_path}  # List files in the directory to see if the file is present with a different name
    else:
        print(f"Directory NOT found: {dir_path}")
        print("The dataset might not have been extracted correctly.")

In [ ]:
#Install Necessary Packages
!pip install tensorflow torch torchvision

In [ ]:
#Data Preprocessing
!pip install pydicom
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pydicom
# Import pydicom to handle DICOM files

# Load dataset CSV

train_labels = pd.read_csv('/content/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv')
# Display the first few rows
print(train_labels.head())

# Preprocess images
def preprocess_image(img_path, target_size=(224, 224)):
    # Read DICOM file
    ds = pydicom.dcmread(img_path)
    img = ds.pixel_array

    # Convert to grayscale if needed
    if img.ndim > 2:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    img = cv2.resize(img, target_size)
    img = cv2.equalizeHist(img)  # Histogram equalization
    img = img / 255.0  # Normalize
    return img

# Verify the file exists
img_path = '/content/rsna-pneumonia-detection-challenge/stage_2_train_images/0004cfab-14fd-4e49-80ba-63afce7341b9.dcm'
if os.path.exists(img_path):
    img = preprocess_image(img_path)
    plt.imshow(img, cmap='gray')
    plt.show()
else:
    print(f"File not found: {img_path}")
    # Investigate why the file is missing - re-download dataset, check extraction, etc.

In [ ]:
#YOLO Model Implementation
# Example code to load and use a pre-trained YOLO model (using TensorFlow)

from tensorflow.keras.applications import YOLO

# Load a pre-trained YOLO model
model = YOLO(weights='imagenet')

# Example prediction
img = preprocess_image('/content/rsna-pneumonia-detection-challenge/stage_2_train_images/sample_image.dcm')
prediction = model.predict(np.expand_dims(img, axis=0))
print(prediction)


In [ ]:
#Faster R-CNN and SSD Implementation
# Example code to load and use pre-trained Faster R-CNN and SSD models (using PyTorch)

import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn, ssd300_vgg16

# Load pre-trained Faster R-CNN model
faster_rcnn_model = fasterrcnn_resnet50_fpn(pretrained=True)
faster_rcnn_model.eval()

# Load pre-trained SSD model
ssd_model = ssd300_vgg16(pretrained=True)
ssd_model.eval()

# Example prediction with Faster R-CNN
img_tensor = transforms.ToTensor()(img)
prediction = faster_rcnn_model([img_tensor])
print(prediction)


In [ ]:
# Ensemble Techniques, Combine predictions from YOLO, Faster R-CNN, and SSD
yolo_prediction = model.predict(np.expand_dims(img, axis=0))
faster_rcnn_prediction = faster_rcnn_model([img_tensor])
ssd_prediction = ssd_model([img_tensor])

# Example of ensemble averaging (adjust based on your predictions)
ensemble_prediction = (yolo_prediction + faster_rcnn_prediction + ssd_prediction) / 3
print(ensemble_prediction)


In [ ]:
# Evaluate Models, Example code for evaluation (adjust as per your evaluation metrics)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

true_labels = [0, 1, 0, 1]  # Example true labels
predicted_labels = [0, 1, 0, 0]  # Example predicted labels

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-score: {f1}")


In [ ]:
#Explainability Methods, Example code for using LIME (adjust based on your implementation)
from lime.lime_image import LimeImageExplainer

explainer = LimeImageExplainer()
explanation = explainer.explain_instance(img, model.predict, top_labels=1, num_samples=1000)

In [ ]:
#Visualize Explainability, Example code for visualizing Grad-CAM (adjust based on your implementation)
import matplotlib.pyplot as plt
from torchvision.models import resnet50
from torchcam.cams import GradCAM

model = resnet50(pretrained=True)
cam = GradCAM(model=model, target_layer='layer4')

# Visualize Grad-CAM heatmap
heatmap = cam(input_tensor)
plt.imshow(heatmap)
plt.show()
